In [1]:
import nni
import numpy as np
import pandas as pd
import torch
import torch.utils.data as Data
from torch import nn
from d2l import torch as d2l
from torch.nn import functional as F
import matplotlib.pyplot as plt
import time 
from sklearn.metrics import r2_score
import time

In [2]:
# 操作数据
train_data1 = pd.read_csv("DoubleBox_Coupler/DoubleBox_train.csv")
train_data1 = train_data1.iloc[:, 1:]
# label_data1 = pd.Series([1]*24000)
# train_data1 = pd.concat([label_data1, train_data1], axis=1)
# train_data1.rename(columns = {0:"label"}, inplace = True)

test_data1 = pd.read_csv("DoubleBox_Coupler/DoubleBox_test.csv")
test_data1 = test_data1.iloc[:, 1:]
# label_data1 = pd.Series([1]*6000)
# test_data1 = pd.concat([label_data1, test_data1], axis=1)
# test_data1.rename(columns = {0:"label"}, inplace = True)

train_data2 = pd.read_csv("DoubleBox_Coupler/Branchline_train.csv")
train_data2 = train_data2.iloc[:, 1:]
# label_data2 = pd.Series([2]*24000)
# train_data2 = pd.concat([label_data2, train_data2], axis=1)
# train_data2.rename(columns = {0:"label"}, inplace = True)

test_data2 = pd.read_csv("DoubleBox_Coupler/Branchline_test.csv")
test_data2 = test_data2.iloc[:, 1:]
# label_data2 = pd.Series([2]*6000)
# test_data2 = pd.concat([label_data2, test_data2], axis=1)
# test_data2.rename(columns = {0:"label"}, inplace = True)
train_data2['w7'] = 0
test_data2['w7'] = 0


train_data2.columns = train_data1.columns
train_data = pd.concat([train_data1,train_data2])

test_data2.columns = test_data1.columns
test_data = pd.concat([test_data1,test_data2])

# test_data = test_data.sample(frac=0.2,axis=0,random_state=2)

# 划分训练特征和测试标签
train_features = torch.tensor(train_data.iloc[:,:14].values, dtype=torch.float32)
test_features = torch.tensor(test_data.iloc[:, :14].values, dtype=torch.float32)
train_labels = torch.tensor(train_data.iloc[:, 14:].values, dtype=torch.float32)
test_labels = torch.tensor(test_data.iloc[:, 14:].values, dtype=torch.float32)

In [3]:
torch_traindata = Data.TensorDataset(train_features, train_labels)
torch_testdata = Data.TensorDataset(test_features, test_labels)

params = {
    'batch': 100,
    'lr': 0.0024305442193705115,
}

BATCH_SIZE = params['batch']

train_dataloader = Data.DataLoader(
    dataset = torch_traindata,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

test_dataloader = Data.DataLoader(
    dataset = torch_testdata,
    batch_size = BATCH_SIZE,
    shuffle = True,
)

optimized_params = nni.get_next_parameter()
params.update(optimized_params)

/home/delta/miniconda3/envs/jiteng3/lib/python3.8/site-packages/nni/runtime/trial_command_channel/standalone.py:34: RuntimeWarning: Running trial code without runtime. Please check the tutorial if you are new to NNI: https://nni.readthedocs.io/en/stable/tutorials/hpo_quickstart_pytorch/main.html
  warnings.warn(warning_message, RuntimeWarning)


In [4]:
device = "cuda:1" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class Reshape1(torch.nn.Module):
    def forward(self, x):
        return x.view(-1, 14)
    
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.reshape = Reshape1()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(14, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 40),      
#             nn.Linear(params['features'], 6)
        )

    def forward(self, x):
        x = self.reshape(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])

Using cuda:1 device


In [6]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    return loss

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            R2_value = r2_score(pred.to("cpu"),y.to("cpu"))
#             correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
#     correct /= size
    return test_loss, R2_value

In [10]:
epochs = 100
test_accuracy = []
train_accuracy = []
R2_accuracy = []
time1 = time.time()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loss = train(train_dataloader, model, loss_fn, optimizer)
    train_accuracy.append(train_loss)
    accuracy, R2_value = test(test_dataloader, model, loss_fn)
    nni.report_intermediate_result(accuracy)
    test_accuracy.append(accuracy)
    R2_accuracy.append(R2_value)
    print(f'Epoch: {t:02d}, Trainloss: {train_loss:.6f}, TestR2: {R2_value:.4f}')
time2 = time.time()
nni.report_final_result(accuracy)


Epoch 1
-------------------------------
[2024-04-01 12:44:37] Intermediate result: 0.03378447766105334  (Index 3)
Epoch: 00, Trainloss: 0.035186, TestR2: -0.0419
Epoch 2
-------------------------------
[2024-04-01 12:44:38] Intermediate result: 0.03160092793405056  (Index 4)
Epoch: 01, Trainloss: 0.031204, TestR2: 0.1507
Epoch 3
-------------------------------
[2024-04-01 12:44:38] Intermediate result: 0.03089409399156769  (Index 5)
Epoch: 02, Trainloss: 0.034967, TestR2: 0.2880
Epoch 4
-------------------------------
[2024-04-01 12:44:38] Intermediate result: 0.02642284131919344  (Index 6)
Epoch: 03, Trainloss: 0.027862, TestR2: 0.4397
Epoch 5
-------------------------------
[2024-04-01 12:44:38] Intermediate result: 0.02647377224639058  (Index 7)
Epoch: 04, Trainloss: 0.026724, TestR2: 0.4493
Epoch 6
-------------------------------
[2024-04-01 12:44:38] Intermediate result: 0.02489839990933736  (Index 8)
Epoch: 05, Trainloss: 0.021155, TestR2: 0.5419
Epoch 7
-------------------------

In [11]:
print("The training time:", round(time2-time1, 2), "s")

The training time: 12.38 s


In [12]:
def getModelSize(model):
    param_size = 0
    param_sum = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
        param_sum += param.nelement()
    buffer_size = 0
    buffer_sum = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
        buffer_sum += buffer.nelement()
    all_size = (param_size + buffer_size) / 1024
    print('The model size is：{:.3f}KB'.format(all_size))
    return (param_size, param_sum, buffer_size, buffer_sum, all_size)
    
a, b, c, d, e = getModelSize(model)

The model size is：365.906KB
